In [46]:
import numpy as np
import math

6. Система координат камеры получается из мировой поворотом на <code>45 градусов вокруг оси z и трансляцией на 10 вдоль оси z</code>. Внутренние параметры камеры: <code>fx=fy=400, cx=960, cy=540</code>. Написать программу, которая выводит на экран матрицу проекции камеры и целочисленные координаты пикселя на изображении, соответствующего трехмерной точке с координатами <code>(10, -10, 100)</code> в мировой системе координат.

In [47]:
rt = np.array([[math.cos(math.radians(45)), -math.sin(math.radians(45)), 0, 0],
              [math.sin(math.radians(45)), math.cos(math.radians(45)), 0, 0],
              [0, 0, 1, 10]])

k = np.array([[400, 0, 960],
              [0, 400, 540],
              [0, 0, 1]])

point = np.array([10, -10, 100, 1])

p = k @ rt

p = round(p.tolist(), 2)

n_point = p @ point

n_point = round(n_point.tolist())

print("Матрица проекции камеры:\n")
p


TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
print("Координаты пикселя: \n")

n_point

Координаты пикселя: 



array([1.11257e+05, 5.94000e+04, 1.10000e+02])

7. Используя прямое линейное преобразование (<code>DLT</code>) и <code>SVD</code>, написать программу, которая принимает на вход множество пар двумерных точек, и выводит на экран соответствующую этим парам матрицу гомографии. Написать генератор пар двумерных точек для теста алгоритма, удовлетворяющих заданному заранее преобразованию гомографии.

8. Модифицировать генератор из <code>Задачи 7</code> так, чтобы он выдавал <code>n=1000</code> соответствий, из которых <code>30%</code> соответствуют заданному заранее преобразованию гомографии, а остальные взяты случайно (преобразованию гомографии не удовлетворяют). Реализовать нахождение преобразования гомографии, используя <code>DLT</code> и <code>RANSAC</code>.